# What is this notebook

TODO

In [ ]:
from brightway2 import *

## Basic setup

Start a new project, and install base data

In [10]:
projects.project = "student-project-SimaPro-import"
bw2setup()

Creating default biosphere

Applying strategy: drop_unspecified_subcategories

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.697 sec



Title: Writing activities to SQLite3 database:
  Started: 05/17/2015 23:50:33
  Finished: 05/17/2015 23:50:34
  Total time elapsed: 0.697 sec
  CPU %: 91.200000
  Memory %: 1.265717
Created database: biosphere3
Creating default LCIA methods

Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Wrote 692 LCIA methods with 170915 characterization factors
Creating core data migrations



Also want ecoinvent database

In [11]:
ei = SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA Documents/Ecoinvent/3.1/cutoff/datasets", 
    "ecoinvent 3.1 cutoff"
)
ei.apply_strategies()
ei.write_database()

Extracting ecospold2 files:
0%                          100%
[##############################] | ETA[sec]: 0.000 | Item ID: fff527b1-0fe4-4
Total time elapsed: 125.184 sec


Title: Extracting ecospold2 files:
  Started: 05/17/2015 23:51:58
  Finished: 05/17/2015 23:54:04
  Total time elapsed: 125.184 sec
  CPU %: 86.800000
  Memory %: 11.837053
Extracted 11301 datasets in 127.29 seconds
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 73.233 sec



Title: Writing activities to SQLite3 database:
  Started: 05/17/2015 23:54:07
  Finished: 05/17/2015 23:55:20
  Total time elapsed: 73.233 sec
  CPU %: 83.500000
  Memory %: 12.867594
Created database: ecoinvent 3.1 cutoff


## First attempt

In [2]:
fp = "/Users/cmutel/Downloads/caes.CSV"
sp = SimaProCSVImporter(fp, name="CAES")
sp.statistics()

DuplicateName: Parameter name pi is a built-in symbol

The formula parser throws an error - it has a certain number of symbols already defined, and when one of those symbol names is used as a variable it raises an error. In this case, we already have $\pi$ defined, so we can manually delete it from the CSV file.

In [3]:
fp = "/Users/cmutel/Downloads/caes-no-pi.CSV"
sp = SimaProCSVImporter(fp, name="CAES")
sp.statistics()

Extracted 6 unallocated datasets in 0.04 seconds
6 datasets
1331 exchanges
1331 unlinked exchanges
  Type biosphere: 1297 unique unlinked exchanges
  Type production: 6 unique unlinked exchanges
  Type technosphere: 23 unique unlinked exchanges


(6, 1331, 1331)

## Apply default strategies

In [4]:
sp.apply_strategies()
sp.statistics()

Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: link_iterable_by_fields
6 datasets
1331 exchanges
26 unlinked exchanges
  Type biosphere: 5 unique unlinked exchanges
  Type technosphere: 21 unique unlinked exchanges


(6, 1331, 26)

## Look at each undefined exchange

In [6]:
for exc in sp.unlinked:
    print(exc['name'])

Air compressor, screw-type compressor, 300kW {GLO}| market for | Alloc Def, U
Electricity, medium voltage {CH}| market for | Alloc Def, U
Gas turbine, 10MW electrical {GLO}| market for | Alloc Def, U
Air
Concrete block {GLO}| market for | Alloc Def, U
Reinforcing steel {GLO}| market for | Alloc Def, U
Clay
Water, unspecified natural origin/m3
Gas turbine, 10MW electrical {RER}| construction | Alloc Def, U
Gas turbine, 10MW electrical {RoW}| construction | Alloc Def, U
Tap water {Europe without Switzerland}| market for | Alloc Def, U
Phosphoric acid, fertiliser grade, without water, in 70% solution state {GLO}| market for | Alloc Def, U
Nitric acid, without water, in 50% solution state {GLO}| market for | Alloc Def, U
Sodium hydroxide, without water, in 50% solution state {GLO}| market for | Alloc Def, U
Nickel, 99.5% {GLO}| market for | Alloc Def, U
Boric acid, anhydrous, powder {GLO}| market for | Alloc Def, U
Metal coating facility {GLO}| market for | Alloc Def, U
Electricity, medium

## Fix links to ecoinvent 3.1

This is a two-step process. First, we have to change the names to ones that ecoinvent uses (no, there is not ecoinvent process `Steel, chromium steel 18/8 {GLO}| market for | Alloc Def, U`!).

Then we can try to link against the Ecoinvent 3.1 database.

In [8]:
sp.migrate("simapro-ecoinvent-3")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


Because we are importin from SimaPro, the categories are screwed up (i.e. ecoinvent categories aren't cleanly imported). Tell the matching algorithm to ignore categories and only use product, reference product, unit, and location.

In [12]:
import functools
from bw2io.strategies import link_iterable_by_fields

sp.apply_strategy(functools.partial(
        link_iterable_by_fields, 
        other=Database("ecoinvent 3.1 cutoff"),
        kind="technosphere",
        fields=["reference product", "name", "unit", "location"]
))
sp.statistics()

Applying strategy: link_iterable_by_fields
6 datasets
1331 exchanges
5 unlinked exchanges
  Type biosphere: 5 unique unlinked exchanges


(6, 1331, 5)

## What is left?

In [13]:
for exc in sp.unlinked:
    print(exc['name'])

Air
Clay
Water, unspecified natural origin/m3
Water/m3
Water, SK


## What to do when the matching is "good enough"

In this case, we are going to ignore these missing exchanges. We do this (naturally) through another strategy, thoough this one has a shortcut:

In [14]:
sp.drop_unlinked()

Oops, we got a complaint. But we proceed anyway:

In [15]:
sp.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked


## Write the database

In [16]:
sp.write_database()

Writing activities to SQLite3 database:
0%  100%
[######] | ETA[sec]: 0.000 
Total time elapsed: 17.039 sec


Title: Writing activities to SQLite3 database:
  Started: 05/18/2015 00:03:48
  Finished: 05/18/2015 00:04:05
  Total time elapsed: 17.039 sec
  CPU %: 5.600000
  Memory %: 13.854790
Created database: CAES
